In [1]:
# !pip install openpyxl

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
# Read the reference CSV (the one with IDs to filter by)
reference_csv = pd.read_csv(r"C:\Users\Khizer Zakir\Downloads\Brick_Kilns_IND-Main_coal_dedup_50m.csv")
print("Reference CSV loaded:")
print(f"Shape: {reference_csv.shape}")
print(f"Columns: {reference_csv.columns.tolist()}")
print(reference_csv.head())

# Get unique IDs from reference CSV
id_column = input("Enter the ID column name in reference CSV: ").strip()
reference_ids = reference_csv[id_column].unique()
print(f"\nUnique IDs in reference CSV: {len(reference_ids)}")
print(f"Sample IDs: {reference_ids[:10]}")

Reference CSV loaded:
Shape: (22459, 18)
Columns: ['id', 'name', 'lat', 'lon', 'type', 'fuel', 'region', 'country', 'status', 'capacity', 'emfpm10', 'pm10', 'emfpm25', 'pm25', 'emfso2', 'so2', 'emfnox', 'nox']
      id          name        lat        lon  type  fuel         region  \
0  BKI-1  Brick Kiln 1  26.162939  81.700951  FCBK  coal  Uttar Pradesh   
1  BKI-2  Brick Kiln 2  25.694000  85.157951  FCBK  coal          Bihar   
2  BKI-3  Brick Kiln 3  25.838212  85.469401  FCBK  coal          Bihar   
3  BKI-4  Brick Kiln 4  25.666453  85.490873  FCBK  coal          Bihar   
4  BKI-5  Brick Kiln 5  25.713344  85.408891  FCBK  coal          Bihar   

  country     status capacity  emfpm10     pm10  emfpm25      pm25  emfso2  \
0   India  operating   11,635      9.7  0.11286      6.8  0.079118     4.6   
1   India  operating   11,635      9.7  0.11286      6.8  0.079118     4.6   
2   India  operating   11,635      9.7  0.11286      6.8  0.079118     4.6   
3   India  operating   11,6

In [11]:
# Read target file (CSV or XLSX) and filter by IDs from reference
import os
from pathlib import Path

target_file_path = input("\nEnter the path to the target CSV or XLSX file to filter: ").strip()

# Check file type and load accordingly
if target_file_path.lower().endswith('.xlsx'):
    # For XLSX, show available sheets
    xls_file = pd.ExcelFile(target_file_path)
    print(f"\nAvailable sheets in XLSX: {xls_file.sheet_names}")
    sheet_name = input("Enter the sheet name to load: ").strip()
    target_csv = pd.read_excel(target_file_path, sheet_name=sheet_name)
    file_type = 'xlsx'
else:
    # For CSV
    target_csv = pd.read_csv(target_file_path)
    file_type = 'csv'

print(f"\nTarget file loaded:")
print(f"Shape: {target_csv.shape}")
print(f"Columns: {target_csv.columns.tolist()}")

# Get ID column name in target file
target_id_column = input("Enter the ID column name in target file: ").strip()

# Filter target file to keep only rows with IDs from reference CSV
filtered_csv = target_csv[target_csv[target_id_column].isin(reference_ids)].copy()

print(f"\nFiltered Results:")
print(f"Original shape: {target_csv.shape}")
print(f"Filtered shape: {filtered_csv.shape}")
print(f"Rows kept: {filtered_csv.shape[0]} ({(filtered_csv.shape[0]/target_csv.shape[0]*100):.1f}%)")
print(f"\nFirst few rows of filtered data:")
print(filtered_csv.head())


Available sheets in XLSX: ['Brick_Kilns_IND-Main_coal', 'EF']

Target file loaded:
Shape: (34466, 18)
Columns: ['id', 'lat', 'lon', 'type', 'fuel', 'state', 'country', 'status', 'capacity_tonnes', 'emfpm10', 'pm10_t_yr', 'emfpm25', 'pm25_t_yr', 'emfso2', 'so2_t_yr', 'emfnox', 'Nox_t_yr', 'source']

Filtered Results:
Original shape: (34466, 18)
Filtered shape: (22459, 18)
Rows kept: 22459 (65.2%)

First few rows of filtered data:
      id        lat        lon  type  fuel          state country     status  \
0  BKI-1  26.162939  81.700951  FCBK  coal  Uttar Pradesh   India  operating   
1  BKI-2  25.694000  85.157951  FCBK  coal          Bihar   India  operating   
2  BKI-3  25.838212  85.469401  FCBK  coal          Bihar   India  operating   
3  BKI-4  25.666453  85.490873  FCBK  coal          Bihar   India  operating   
4  BKI-5  25.713344  85.408891  FCBK  coal          Bihar   India  operating   

   capacity_tonnes  emfpm10  pm10_t_yr  emfpm25  pm25_t_yr  emfso2   so2_t_yr  \
0   

In [12]:
# Create filtered folder and save outputs
import os
from pathlib import Path
from shapely.geometry import Point

# Create filtered folder in the same directory as target file
target_dir = os.path.dirname(target_file_path)
filtered_folder = os.path.join(target_dir, "filtered")
os.makedirs(filtered_folder, exist_ok=True)

# Get base filename without extension
base_filename = Path(target_file_path).stem

# Save in original format
if file_type == 'xlsx':
    # Save as XLSX with _filtered suffix
    filtered_xlsx_path = os.path.join(filtered_folder, f"{base_filename}_filtered.xlsx")
    filtered_csv.to_excel(filtered_xlsx_path, index=False, engine='openpyxl')
    print(f"[OK] Filtered XLSX saved to: {filtered_xlsx_path}")
else:
    # CSV -> Convert to GeoJSON
    # Get lat/lon columns from user
    filtered_csv_path = os.path.join(filtered_folder, f"{base_filename}_filtered.csv")
    filtered_csv.to_csv(filtered_csv_path, index=False)
    print(f"[OK] Filtered CSV saved to: {filtered_csv_path}")
    print("\nAvailable columns: ", filtered_csv.columns.tolist())
    lat_col = input("Enter latitude column name: ").strip()
    lon_col = input("Enter longitude column name: ").strip()
    
    # Create geometry from lat/lon
    geometry = [Point(xy) for xy in zip(filtered_csv[lon_col], filtered_csv[lat_col])]
    gdf = gpd.GeoDataFrame(filtered_csv, geometry=geometry, crs='EPSG:4326')
    
    # Save as GeoJSON
    filtered_geojson_path = os.path.join(filtered_folder, f"{base_filename}_filtered.geojson")
    gdf.to_file(filtered_geojson_path, driver='GeoJSON')
    print(f"[OK] CSV converted to GeoJSON and saved to: {filtered_geojson_path}")

print(f"\n[OK] All files saved to: {filtered_folder}")

[OK] Filtered XLSX saved to: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\filtered\Brick_Kilns_IND-Main_coal_filtered.xlsx

[OK] All files saved to: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\filtered


In [8]:
# read the geojson back to verify
gdf_check = gpd.read_file(filtered_geojson_path)
print(f"\nVerification - Loaded GeoJSON shape: {gdf_check.shape}")
print(gdf_check.head())


Verification - Loaded GeoJSON shape: (22459, 19)
      id        lat        lon  type     fuel          state country  \
0  BKI-1  26.162939  81.700951  FCBK  biomass  Uttar Pradesh   India   
1  BKI-2  25.694000  85.157951  FCBK  biomass          Bihar   India   
2  BKI-3  25.838212  85.469401  FCBK  biomass          Bihar   India   
3  BKI-4  25.666453  85.490873  FCBK  biomass          Bihar   India   
4  BKI-5  25.713344  85.408891  FCBK  biomass          Bihar   India   

      status capacity_tonnes   emfpm10  pm10_t_yr   emfpm25  pm25_t_yr  \
0  operating        7,504.58  0.000747   5.608919  0.000404   3.031848   
1  operating        7,504.58  0.000747   5.608919  0.000404   3.031848   
2  operating        7,504.58  0.000747   5.608919  0.000404   3.031848   
3  operating        7,504.58  0.000747   5.608919  0.000404   3.031848   
4  operating        7,504.58  0.000747   5.608919  0.000404   3.031848   

    emfso2   so2_t_yr   emfnox  Nox_t_yr source                   geomet

In [13]:


folder = r"D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln"
import os
# all the csv files in a folder to geojson

for file in os.listdir(folder):
    if file.endswith(".csv"):
        file_path = os.path.join(folder, file)
        print(f"Processing file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Columns available: {df.columns.tolist()}")
        lat_col = input("Enter latitude column name: ").strip()
        lon_col = input("Enter longitude column name: ").strip()
        
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
        
        geojson_path = os.path.join(folder, f"{Path(file).stem}.geojson")
        gdf.to_file(geojson_path, driver='GeoJSON')
        print(f"[OK] Converted to GeoJSON and saved to: {geojson_path}\n")


Processing file: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\Brick_kilns_BAN-Main_biomass.csv
Columns available: ['id', 'lat', 'lon', 'type', 'fuel', 'state', 'country', 'status', 'capacity_tonnes', 'emfpm10', 'pm10_t_yr', 'emfpm25', 'pm25_t_yr', 'emfso2', 'so2_t_yr', 'emfnox', 'Nox_t_yr', 'source']
[OK] Converted to GeoJSON and saved to: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\Brick_kilns_BAN-Main_biomass.geojson

Processing file: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\Brick_kilns_BAN-Main_coal.csv
Columns available: ['id', 'lat', 'lon', 'type', 'fuel', 'state', 'country', 'status', 'capacity_tonnes', 'emfpm10', 'pm10_t_yr', 'emfpm25', 'pm25_t_yr', 'emfso2', 'so2_t_yr', 'emfnox', 'Nox_t_yr', 'source']
[OK] Converted to GeoJSON and saved to: D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Da

In [15]:
# read geojson

try_1 = gpd.read_file(r"D:\APAD_oxford\Final_Dataset_Folder\dataset\APAD-Asset-Data\Asset Data\IGP\Asset_Data_IGP\Brick Kiln\Brick_Kilns_PK-Main_coal.geojson")
print(try_1.shape)
print(try_1.head())

(11272, 19)
      id        lat        lon  type  fuel  state   country     status  \
0  BKP-1  26.133092  68.565789  FCBK  coal  Sindh  Pakistan  operating   
1  BKP-2  26.131069  68.559271  FCBK  coal  Sindh  Pakistan  operating   
2  BKP-3  26.139061  68.564912  FCBK  coal  Sindh  Pakistan  operating   
3  BKP-4  26.132216  68.572884  FCBK  coal  Sindh  Pakistan  operating   
4  BKP-5  26.129035  68.576938  FCBK  coal  Sindh  Pakistan  operating   

  capacity_tonnes  emfpm10  pm10_t_yr  emfpm25  pm25_t_yr  emfso2   so2_t_yr  \
0        7,498.13        0  72.731813   0.0068   50.98725  0.0046  34.491375   
1        7,498.13        0  72.731813   0.0068   50.98725  0.0046  34.491375   
2        7,498.13        0  72.731813   0.0068   50.98725  0.0046  34.491375   
3        7,498.13        0  72.731813   0.0068   50.98725  0.0046  34.491375   
4        7,498.13        0  72.731813   0.0068   50.98725  0.0046  34.491375   

   emfnox   Nox_t_yr source                   geometry  
0  0.